In [70]:
#necessary imports
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer #, ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB

In [78]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path): #os.walk(path)function finds all the files in a directory
        for filename in filenames:
            path = os.path.join(root, filename) #builds up full pathname for each file in the directory 
            
            #inBody = False
            inBody = True
            lines = []
            f = io.open(path, 'r', encoding='latin-1') #then reads it
            for line in f:  #this for block makes sure to skip the headlines for each email
                if inBody:
                    lines.append(line)
                elif line == '\n': 
                    inbody = True
                '''we're looking for 1st blank space, 
                we know after the blank space-it's the original mail or message body, 
                and before that everything is a bunch of header,which I dont want to train my spam classifier on.
                so condition is given in elif'''
            f.close()
            message = "\n".join(lines)
            yield path, message
'''This function iterates every file in a directory, 
reads it,skips the headlines and lists only the email body. 
So this function gives me both full path of each email file and email or message body.''' 

'This function iterates every file in a directory, \nreads it,skips the headlines and lists only the email body. \nSo this function gives me both full path of each email file and email or message body.'

In [83]:
def DataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message':message, 'class':classification})
        index.append(filename)
    return DataFrame(rows, index=index)
# this function says I have a path to a directory and it has already been given a classification as spam or not spam.

data = DataFrame({"message":[], "class":[]}) #creates a mini database consisted of message and class columns
data = data.append(DataFrameFromDirectory('/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/spam', 'spam'))
data = data.append(DataFrameFromDirectory('/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/ham', 'not spam'))

/home/pritom/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [84]:
data.head() #pandas head function, shows first 5 lines of data
#data.head(n) #pandas head function, shows first n lines of data

,class,message
/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/spam/00195.0a543c2780491168160570bb6708af86,spam,From dz524878@caramail.com Mon Sep 2 16:27:2...
/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/spam/00488.29e96da757cc5566c848833e26abdd65,spam,From beautyinfufuxxxmeb13mxy@aol.com Tue Oct ...
/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/spam/00467.5b733c506b7165424a0d4a298e67970f,spam,From InkjetDeals@acsmsupplies.com Thu Sep 26 ...
/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/spam/00332.580b62752adefb845db173e375271cb5,spam,Return-Path: ler@lerami.lerctr.org\n\nDelivery...
/home/pritom/Desktop/ML datasets/spam or ham email datasets/emails/spam/00257.5c8ef87f8b11d2515df71a7fe46a70b6,spam,Return-Path: ler@lerami.lerctr.org\n\nDelivery...


In [85]:
vectorizer = CountVectorizer() #stop_words=None, input=file
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB() #model
targets = data["class"].values
classifier.fit(counts, targets) #model fitting

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [86]:
examples = ["Free viagra now!!", "Hi miha, how about a practise tomorrow?"]
examples_counts = vectorizer.transform(examples) #words are positioned in an array/list by their occurance and frequency
predictions = classifier.predict(examples_counts)
predictions

array(['spam', 'not spam'], dtype='<U8')